In [2]:
from IPython.display import HTML, display
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

'en_US.UTF-8'

In [3]:
def calculateNodes(stake, rewardrate, targetDays, startNodes):
    currentNode = startNodes
    days = 0
    while (True):
        cycleTime = stake / rewardrate / currentNode
        days += cycleTime
        if (days > targetDays):
            break
        currentNode += 1
    return { "totalNodes": currentNode, "newNodes": currentNode -  startNodes}

In [4]:
def nice(number):
    return locale.format_string('%d', int(number), grouping=True)

In [5]:
def getCostsTitles(costs):
    titles = []
    for title in costs.keys():
        titles.append(f'{title} costs USD')
    return titles

In [6]:
def getCostsData(costs, rewardsUSD):
    values = []
    for factor in costs.keys():
        values.append(nice(rewardsUSD * costs[factor]))
    return values

In [7]:
def getDividendsTitles(dividends):
    titles = []
    for title in dividends.keys():
        titles.append(f'{title} dividends USD')
    return titles

In [8]:
def getDividendsData(dividends, dividendsFraction, rewardsUSD):
    values = []
    for factor in dividends.keys():
        values.append(nice(rewardsUSD * dividendsFraction * dividends[factor]))
    return values

In [9]:
def calculateYear(year, startNodes, params):
    costsFraction = sum(params["costs"].values())
    growthFraction = params["years"][year]["growthFraction"] - sum(params["costs"].values())
    dividendsFraction = 1 - growthFraction - costsFraction
    
    result = calculateNodes(
        params["initial"]["stake"] * (1 / growthFraction),
        params["years"][year]["rewardrate"], 
        params["years"][year].get("days", 365), 
        startNodes)
    
    stakePOKT = result["totalNodes"] * params["initial"]["stake"]
    stakeUSD = stakePOKT * params["years"][year]["price"]
    
    rewardsPOKT = result["newNodes"] * params["initial"]["stake"] * (1 / growthFraction)
    rewardsUSD = rewardsPOKT * params["years"][year]["price"]
    
    costsData = getCostsData(params["costs"], rewardsUSD)
    dividendsData = getDividendsData(params["dividends"], dividendsFraction, rewardsUSD)
    
    
    return [
        year, 
        result["newNodes"], 
        result["totalNodes"], 
        nice(stakePOKT),
        nice(rewardsPOKT),
        nice(stakeUSD),
        nice(rewardsUSD),
    ] + costsData + dividendsData

In [10]:
def calculateYears(params):
    data = [
        [
        'year', 
        'newNodes', 
        'totaNodes',
        'stake POKT',
        'rewards POKT', 
        'stake USD',
        'rewards USD'
        ] + getCostsTitles(params["costs"]) + getDividendsTitles(params["dividends"])
    ]
    
    for year in params["years"].keys():
        startNodes = data[-1][2] if len(data) > 1 else params["initial"]["startNodes"]
        data.append(calculateYear(year, startNodes, params))
        
    display(HTML(
        '<table style="font-size: 18px"><tr>{}</tr></table>'.format(
            '</tr><tr>'.join(
                '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in data)
        )
    ))

In [14]:
calculateYears({
    "initial": {
        "stake": 60000,
        "startNodes": 250,
    },
    "costs": {
        "30%": .3,
        "20%": .2,
    },
    "dividends": {
        "25%": .25,
    },
    "years": {
        2022: { "growthFraction": 1.0, "rewardrate": 80, "price": 0.06, "days": 90},
        2023: { "growthFraction": 0.9, "rewardrate": 80, "price": 0.01},
        2024: { "growthFraction": 0.9, "rewardrate": 80, "price": 0.05},
        2025: { "growthFraction": 0.9, "rewardrate": 80, "price": 0.10},
        2026: { "growthFraction": 0.9, "rewardrate": 80, "price": 0.10},
        2027: { "growthFraction": 0.5, "rewardrate": 80, "price": 0.10},
        2028: { "growthFraction": 0.5, "rewardrate": 80, "price": 0.10},
        2029: { "growthFraction": 0.5, "rewardrate": 80, "price": 0.10},
        2030: { "growthFraction": 0.5, "rewardrate": 80, "price": 0.10},
        2031: { "growthFraction": 0.5, "rewardrate": 80, "price": 0.20},
        2032: { "growthFraction": 0.5, "rewardrate": 80, "price": 0.20},
        2033: { "growthFraction": 0.5, "rewardrate": 80, "price": 0.20},
    }
})

ZeroDivisionError: float division by zero